In [2]:
# from neuralprophet import NeuralProphet
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from utils import generate_lagged_features, scale_data
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from utils import mape, gmrae, smape_adjusted, nrmse


from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

/Users/pantelisdogoulis/miniforge3/envs/energy-forecasting/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
path = './data.csv'
data = pd.read_csv(path, parse_dates=["datetime"])
data.set_index('datetime', inplace=True)
data["AP"] = data["AP"].astype("category")
data["AP"] = data["AP"].cat.codes

In [ ]:
N_LAGS = 5
N_FORECASTS = 4 # (FORECASTS-1)

In [164]:
X_features = [f'lag{i}' for i in range(N_FORECASTS+1,N_FORECASTS+1+N_LAGS)]
X_features.reverse()
y_features = [f'lag{i}' for i in range(0, N_FORECASTS+1)]
y_features.reverse()

In [167]:
lagged_df = pd.DataFrame()

for customer_id in tqdm(data.AP.unique()):
    customer = data[data['AP']==customer_id]
    generate_lagged_features(data, 'ENERGY', N_LAGS + N_FORECASTS)
    customer.dropna(inplace=True)
    lagged_df = pd.concat((lagged_df, customer))
lagged_df = lagged_df.rename(columns={
                            'scaled':'lag0',})
lagged_df = lagged_df.rename(columns={'ENERGY_MW': 'lag0'})
lagged_df.dropna(inplace=True)

test = lagged_df[lagged_df.index==2021]
train = lagged_df[lagged_df.index.isin([2015,2016,2017,2018,2019,2020])]

100%|██████████| 3910/3910 [15:31<00:00,  4.20it/s]


In [ ]:
lagged_df.to_csv("./data/lagged_df_kw.csv")

In [169]:
test = lagged_df[lagged_df.index.year==2021]
train = lagged_df[lagged_df.index.year.isin([2015,2016,2017,2018,2019,2020])]

In [170]:
train['mean'] = train[X_features].mean(axis=1)
train['std'] = train[X_features].std(axis=1)
train['skew'] = train[X_features].skew(axis=1)
train['kurtosis'] = train[X_features].kurtosis(axis=1)

test['mean'] = test[X_features].mean(axis=1)
test['std'] = test[X_features].std(axis=1)
test['skew'] = test[X_features].skew(axis=1)
test['kurtosis'] = test[X_features].kurtosis(axis=1)

In [171]:
ad_features = ['AP', 'mean', 'std', 'skew', 'sin_dayofyear', 'cos_dayofyear', 'sin_month', 'cos_month', 'sin_week', 'cos_week', 'sin_season', 'cos_season'] 
X_features.extend(ad_features)

In [191]:
# TRAIN:

X_train = train[X_features]
y_train = train[y_features]

# hyperparameter tuning
tscv = TimeSeriesSplit(n_splits=3)
reg = MultiOutputRegressor(LGBMRegressor(n_estimators=400, max_depth=8, min_child_samples=20, reg_lambda=0.1, min_split_gain=20, num_leaves=31))
params = dict(estimator__n_estimators=[500,1000], estimator__max_depth=[5,12,20], \
  estimator__min_child_samples=[20,50,100], estimator__min_split_gain=[10,20,40], estimator__num_leaves=[30,40])
gs = RandomizedSearchCV(estimator=reg, param_distributions=params, cv=tscv, random_state=42)
# fitting
gs.fit(X_train,  y_train)
# selecting best estimator 
reg = gs.best_estimator_

print('finished training')

finished training


In [192]:
metrics_df = pd.DataFrame()
y_pred_df = pd.DataFrame()


for customer_id in tqdm(test.AP.unique()):
    customer_test = test[test['AP']==customer_id]

    customer_test['mean'] = customer_test[X_features].mean(axis=1)
    customer_test['std'] = customer_test[X_features].std(axis=1)
    customer_test['skew'] = customer_test[X_features].skew(axis=1)
    customer_test['kurtosis'] = customer_test[X_features].kurtosis(axis=1)

    X_test = customer_test[X_features]
    y_test = customer_test[y_features]

    y_pred = reg.predict(X_test)

    # y_pred = pd.DataFrame(y_pred)
    # y_pred.columns = y_features
    # y_pred.index = y_test.index

    # NAIVE
    y_naive = pd.DataFrame(X_test[['lag5']*5])
    y_naive.index = X_test.index
    y_naive.columns = y_test.columns
    
    mse = mean_squared_error(y_test.values, y_pred)
    mae = mean_absolute_error(y_test.values, y_pred)
    mape_sk = mean_absolute_percentage_error(y_test.values, y_pred)
    r_sq = r2_score(y_test.values, y_pred)
    # nrmse_value = nrmse(y_test.values, y_pred.values, y_naive.values)
    mape_value = np.mean(mape(y_test.values, y_pred))
    smape_value = np.mean(smape_adjusted(y_test.values, y_pred))
    nrmae_value = mean_absolute_error(y_test, y_pred) / mean_absolute_error(y_test, y_naive)
    # mape_value = mape(y_test.values, y_pred.values)
    # gmrae_value = gmrae(y_test.values, y_pred.values, naive.values)
    # smape_value = smape_adjusted(y_test.values, y_pred.values)

    metrics = np.concatenate((np.reshape(customer_id, (1,1)),np.reshape(mse, (1,1)), \
    np.reshape(mae, (1,1)), np.reshape(mape_sk, (1,1)), np.reshape(r_sq, (1,1)),\
    np.reshape(mape_value, (1,1)), np.reshape(smape_value, (1,1)), np.reshape(nrmae_value, (1,1)) ), axis=1)
    metrics = pd.DataFrame(metrics)
    metrics.columns = ['AP', 'mse', 'mae', 'mape_sk', 'r_sq', 'mape', 'smape', 'nrmae']

    metrics_df = pd.concat((metrics_df, metrics), axis=0)
    
    customer_id_array = np.full((39,1), customer_id)
    y_pred = np.concatenate((y_pred, customer_id_array),1)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['lag4', 'lag3', 'lag2', 'lag1', 'lag0', 'AP']
    y_pred.index = y_test.index

    y_pred_df = pd.concat((y_pred_df, y_pred), axis=0)

100%|██████████| 3910/3910 [01:01<00:00, 63.10it/s]
